In [1]:
pretty_print("in the name of ALLAH")

'in the name of ALLAH'

In [2]:
from sage.sat.boolean_polynomials import solve as solve_sat
from sage.rings.polynomial.multi_polynomial_sequence import PolynomialSequence
import re
import pdb

In [3]:
# It’s possible to compile code in a notebook cell with Cython. For this you need to load the Cython magic:
#Then you can define a Cython cell by writing %%cython on top of it. Like this:
# %%cython
# cython codes
%load_ext cython

In [4]:
%%sh
pwd
#ls -la
make

/home/hosein/Desktop/KeeLoq
gcc -g -Wall -o main  main.c keeloq.c speed.c polygen.c


In [5]:
#%%sh
#./main

In [6]:
%%sh
pwd

/home/hosein/Desktop/KeeLoq


In [7]:
%%cython -I /home/hosein/Desktop/KeeLoq

from libc.stdint cimport uint32_t, uint64_t
from libc.stdlib cimport calloc, free
    
cdef extern from "keeloq.c":
    void keeloq_encrypt(uint64_t *key, uint32_t *plaintext, uint32_t *ciphertext, int nrounds)
    void keeloq_decrypt(uint64_t *key, uint32_t *plaintext, uint32_t *ciphertext, int nrounds)
    
cdef extern from "polygen.c":
    ctypedef struct polys:
        char poly[300]
    ctypedef struct polyequations:
        polys *eqs
        uint64_t number_of_eqs
    polyequations polynomials(uint32_t *plains, uint32_t *ciphers, int r, int number_of_plains)

def keeloq_enc(k, p, r):
    cdef uint64_t k1
    cdef uint32_t p1
    cdef uint32_t c1
    k1, p1 = k, p
    keeloq_encrypt(&k1, &p1, &c1, r)
    return c1

def keeloq_dec(k, c, r):
    cdef:
        uint64_t k1
        uint32_t p1
        uint32_t c1
    k1, c1 = k, c
    keeloq_decrypt(&k1, &p1, &c1, r)
    return p1
    
def keeloq_polys(ptexts, ctexts, r, number_of_plains):
    cdef int l = len(ptexts)
    cdef:
        uint32_t *plains = <uint32_t *> calloc(l, sizeof(uint32_t))
        uint32_t *ciphers = <uint32_t *> calloc(l, sizeof(uint32_t)) 
        polyequations equations
    for i in range(l):
        plains[i] = ptexts[i]
        ciphers[i] = ctexts[i]    
    equations = polynomials(plains, ciphers, r, number_of_plains)
    st = []
    for i in range(equations.number_of_eqs):
        st.append(equations.eqs[i].poly)
    free(plains)
    free(ciphers)
    return st

In [8]:
k = 0x5cec6701b79fd949
p = 0xf741e2db
r = 528
c = keeloq_enc(k, p, r)
hex(c)

'0xe44f4cdf'

In [9]:
@parallel
def paralle_encrypt(p, r):
    k = 0x21537612
    # = randint(0, 10^7)
    return keeloq_enc(k, p, r)

In [10]:
r = 528
number_of_plains = 2^16
plains = [randint(0, 2^32 - 1) for _ in range(number_of_plains)]
inputs = [(plains[i], 2) for i in range(len(plains))]

In [11]:
%time ciphers = paralle_encrypt(inputs)

CPU times: user 27 µs, sys: 3 µs, total: 30 µs
Wall time: 42.9 µs


In [12]:
k = 0x21537612
ciphers = [0]*number_of_plains
%time
for i in range(number_of_plains):
    ciphers[i] = keeloq_enc(k, plains[i], r)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 14.1 µs


In [13]:
k = 0x5cec6701b79fd949
r = 32
number_of_plains = 3
plains = [randint(0, 2^32 - 1) for _ in range(number_of_plains)]
ciphers = [0]*number_of_plains
for i in range(number_of_plains):
    ciphers[i] = keeloq_enc(k, plains[i], r)
%time temp = keeloq_polys(plains, ciphers, r, number_of_plains)
vrs = []
for f in temp:
    terms = re.split(' \+ |\*', f)
    for v in terms:
        if (not v.isdigit()):
            vrs.append(v)
vrs = list(set(vrs))
R = BooleanPolynomialRing(len(vrs), vrs)
%time ps = map(R, temp)
ps = PolynomialSequence(ps)

CPU times: user 1.36 ms, sys: 0 ns, total: 1.36 ms
Wall time: 1.37 ms
CPU times: user 2.43 s, sys: 0 ns, total: 2.43 s
Wall time: 2.44 s


In [14]:
%time sls = solve_sat(ps, n = infinity, s_verbosity = 4)
#print(sls)
print("number of solutions : %d" % len(sls))

CPU times: user 4.34 s, sys: 17.4 s, total: 21.8 s
Wall time: 21.9 s
number of solutions : 1


In [15]:
key_sol = dict()
kindex = []
for i in range(64):
    kname = 'k_%d' % i    
    if kname in R.variable_names():        
        kname = R(kname)
        kval = sls[0].get(kname)
        if kval != None:
            kindex.append(i)            
            key_sol[kname] = kval

In [16]:
original_key = bin(k)[2:].zfill(64)
original_key = list(original_key)
original_key.reverse()
equi_flag = True
for i in kindex:
    #pdb.set_trace()
    if original_key[i] != str(key_sol[R("k_%d" % i)]):        
        equi_flag = False
        break
if (equi_flag == True):
    print("C0ngratulation! The extracted key matches with the original key :)")
else:
    print("There is mismatch :(")

C0ngratulation! The extracted key matches with the original key :)
